Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
    http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.  

# MONAI Label - Bring Your Own Data

***Bring Your Own Data to MONAI Label with Pancreas and Tumor Segmentation***

In this notebook, we provide a step-by-step setup of using the local dataset inside Docker, such as MONAI Toolkit for MONAI Label. The monaibundle app with pancreas and tumor segmentation is demonstrated. The tutorial shows how local CT scans are mounted and loaded to MONAI Label server. With the released `monai_pancreas_ct_dints_segmentation` bundle and 3D Slicer as the client viewer, we provide detailed guidance on auto segmentation, annotation edit, training and active learning. 


**monai_pancreas_ct_dints_segmentation**

This bundle uses a neural architecture search algorithm for volumetric (3D) segmentation of the pancreas and pancreatic tumor from CT images.

The inference and training require at least 16GB-memory GPUs.


**Modality:** CT

The method framework for pancreas and tumor segmentation with MONAI Label shown in 3D Slicer

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/dints.png" alt="drawing" width="700"/></p>
<p align = "center">This model is trained using the neural network model from the neural architecture search algorithm, DiNTS.</p>



## Setup environment

### Prerequisites
- **Install MONAI Label** weekly preview release: 

In [ ]:
!pip install monailabel

## 1. Start Docker (Toolkit) with Local Data
Instead of using sample data provided by tutorials and downloaded by MONAI API, we can use local data to test and train deep learning algorithms with MONAI Label and MONAI Core.

Please go to Docker README such as MONAI Toolkit **Readme.md** for details on mounting the data directory, the local data directory needs to be mounted at starting Toolkit docker. Specifically, use the below command and mount data to the toolkit docker file system.

In this tutorial, we show simple steps to create a local dataset with MSD Task07 Pancreas and tumor data in the local machine and mount the data to Toolkit. 

### 1.1 Prepare Pancreas CT scans as Local Data

Create a directory and prepare a dataset, in this case, we use MSD Pancreas and tumor segmentation dataset. The tutorial shows the dataset is downloaded in the local machine's directory, then mount to use in MONAI Toolkit.

In a local host machine, execute the following command to prepare the pancreas CT dataset.

```bash
mkdir ~/workspace
monailabel datasets --download --name Task07_Pancreas --output ~/workspace
```

To mount a user's local data directory, the users can use `-v` to mount the drive(s) and override the default data directory environment variable `MONAI_DATA_DIRECTORY` used by many notebooks.
For example:

```bash
docker run  --gpus all -it --rm --ipc=host --net host \
            -v ~/workspace/Task07_Pancreas:/opt/toolkit/Task07_Pancreas \
            -e MONAI_DATA_DIRECTORY=/opt/toolkit/Task07_Pancreas \
            nvcr.io/nvidia/clara/monai-toolkit \
            /opt/docker/runtoolkit.sh
```

**Note:** As the docker run command needs to start Toolkit over, you may need to restart Toolkit container with the prepared local data and continue the tutorial from Section 2 

## 2. Start MONAI Label - monaibundle app 

In this tutorial, we use the monaibundle app for demonstrating the bundle models with a segmentation task.

### 2.1 Get the monaibundle App
MONAI Label provides the API to fetch the sample app, type the `moniabundle` as the name and specify the saved directory path `apps`.


In [1]:
!monailabel apps --download --name monaibundle --output apps

Using PYTHONPATH=/opt:

monaibundle is copied at: /opt/toolkit/tutorials/monai/monailabel/apps/monaibundle


### 2.2 Download the Pancreas Bundle
To adapt local data for MONAI Label, users can start MONAI Label server with the following commands.

Save to the monaibundel app **model** directory: monaibundle/model

In [2]:
!mkdir apps/monaibundle/model

Users can automatically download or go to the NGC-released bundle page, for the latest `monai_pancreas_ct_dints_segmentation` bundle:

Users can find information about how to download MONAI bundle in Section 3.2.2 of [Chapter 3](../../chapter-3-using-monai-core.md)

Below is the correct sample directory tree structure for monaibundle app:

```
└── monaibundle
    ├── bin
    ├── __init__.py
    ├── lib
    │   ├── activelearning
    │   │   └── __init__.py
    │   ├── infers
    │   │   └── __init__.py
    │   ├── __init__.py
    │   └── trainers
    │       └── __init__.py
    ├── logs
    │   ├── app.log
    │   └── logging.json
    ├── main.py
    ├── model
    │   └── monai_pancreas_ct_dints_segmentation
    │       ├── configs
    │       │   ├── evaluate.yaml
    │       │   ├── inference.yaml
    │       │   ├── logging.conf
    │       │   ├── metadata.json
    │       │   ├── multi_gpu_train.yaml
    │       │   ├── search.yaml
    │       │   └── train.yaml
    │       ├── docs
    │       │   ├── data_license.txt
    │       │   └── README.md
    │       ├── LICENSE
    │       ├── models
    │       │   ├── model.pt
    │       │   ├── model.ts
    │       │   └── search_code_18590.pt
    │       └── scripts
    │           ├── __init__.py
    │           ├── prepare_datalist.py
    │           └── search.py
    ├── __pycache__
    │   └── main.cpython-39.pyc
    ├── README.md
    └── requirements.txt
```

### 2.3 Starting MONAI Label Server

Specify the bundle name in --conf models <BUNDLENAME> argument. For example monai_pancreas_ct_dints_segmentation, the name needs to match the bundle folder name.
    
In the `monailabel start_server` command, `--app` argument sets the sample-app (monaibundle) path, `--studies` specifies the mounted data
directory path, and `--conf models` loads the bundle config directory. The above command uses `monai_pancreas_ct_dints_segmentation` bundle as an example. The bundle directory should be placed in `model` folder of the monaibundle app.

Please provide a Github authentication token to access the bundle list if there is a rate limit error. 
    
```bash
--conf auth_token <Github Auth Token>
```

Note: monaibundle API supports downloading from both Github and NGC. 

Users can set in the terminal:

```bash
export BUNDLE_DOWNLOAD_SRC="github" # github is set by default

export BUNDLE_DOWNLOAD_SRC="ngc" # if from NGC
```

or in the notebook to specify the download source

`%env BUNDLE_DOWNLOAD_SRC=ngc`

In [ ]:
!monailabel start_server --app apps/monaibundle --studies /opt/toolkit/Task07_Pancreas/imagesTr --conf models pancreas_ct_dints_segmentation

## 3. Pancreas Auto Segmentation with 3D Slicer

Below is a step-by-step tutorial on using 3D Slicer for the auto pancreas and tumor segmentation with the DiNTS bundle. Pre-trained model is provided by the NGC bundle and loaded by MONAI Label server command.


**Use 3D Slicer and MONAI Label Plugin**: the tutorial starts with monaibundle app in the 3D Slicer, please refer to **DevelperGuide** section 2 or the ["Hello World" tutorial](monailabel_radiology_3dslicer.ipynb) for detailed instructions on installing 3D Slicer and MONAI Label plugin.

### 3.1 Select Model and Fetch Data

- In the Slicer, click the MONAI Label plugin with the icon. First, users can select the bundle model loaded with MONAI Label server, such as "**monai_pancreas_ct_dints_segmentation**" in this tutorial. 

Click **next sample** to load image data from Datastore.

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas1.png" alt="drawing" width="700"/></p>


### 3.2 Run Pancreas/Tumor Inference

- The bundle contains a DiNTS pre-trained model weights, on the pancreas and tumor segmentation task, click **run** to run inference.  

- The pre-trained model by DiNTS algorithm is in the bundle's "**models**" folder: "**apps/monaibundle/model/monai_pancreas_ct_dints_segmentation/models/model.pt**".
- Users can move the axis and slices to view the correct location of segmentation.

Users can monitor the logs in the above MONAI Label start server cell.

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas2.png" alt="drawing" width="700"/></p>

### 3.3 Edit Annotations 

- Users can edit the inference labels manually using Segment Editor after auto inference. 

#### Corrections with Segment Editor:

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas3.png" alt="drawing" width="700"/></p>

### 3.4 Submit Label

- Click **Submit Label** to save the ground truth label to the file system. The final annotations will be saved to "**labels/final**" folder in the local dataset, in this tutorial, the annotated label will be saved to "**~/workspace/Task07_Pancreas/imagesTr/labels/final**"

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas4.png" alt="drawing" width="700"/></p>


## 4. Fine-Tune Model with 3D Slicer

MONAI Label offers **active learning** strategies and can interactively fine-tune models. Users can train their models once new annotated labels are saved, with any number of finished labels. MONAI Label server set train and validation split percentage to 0.2 by default. It's recommended to label five or more before running training. Click **Train** button, MONAI Label server will fetch saved final ground truth label and fine-tune the prior model. 

Users can monitor the logs in the MONAI Label server terminal

<p align = "center"><img src="./figures/monailabel_bring_your_own_data/pancreas5.png" alt="drawing" width="700"/></p>

The active learning process will select unlabeled batch data, users can choose strategies in the Slicer UI.

In the second round of labeling, when next image is selected, those trained images will be marked as labeled data, and won't be selected. Active learning strategies such as "first/random" will be used for selecting which unlabeled data. 



## 4. Repeat Until All Data are Annotated and Trained 

Iteratively label and train samples until all unlabeled data are annotated. The model will be fine-tuned with all labeled data at the end. 

For each training loop, the new best metric model will be saved in "**model/model.pt**", in this use case, "**apps/monaibundle/monai_pancreas_ct_dints_segmentation/model/model.pt**" is updated and saved.

## 5. Conclusion

This tutorial demonstrates how to bring your own data to MONAI Label. The pancreas and tumor segmentation with the DiNTS bundle is shown. The step-by-step tutorial on preparing and mounting local data, starting MONAI Label server with the downloaded bundle, auto inference, segmentation editor, submit labels, and training process is introduced. MONAI Label and its apps highlight the robustness of adapting users' local data, and the workflow can be used to extend data and model deployments. 